## scVI data integration for healthy PBMC pilot study (Cai 2020 and Cai 2022)

**Objective**: Run scVI data integration model for healthy PBMCs [Cai 2020 and Cai 2022]


- **Developed by**: Mairi McClean

- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**

- v230321

- Following this tutorial:https://docs.scvi-tools.org/en/stable/tutorials/notebooks/harmonization.html

